In [1]:
import os
%pwd

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow'

In [5]:
from pathlib import Path 
import os 
import zipfile 
import urllib.request as request 
from MLproject.utils.common import read_yaml, get_size, create_directories
from MLproject import logger 
from MLproject.constants import *


In [6]:
class ConfigurationManagerTask:
    def __init__(self, config_filepath = CONFIG_FILE_PATH):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion(self):
        initialize = self.config.data_ingestion

        create_directories([initialize.root_dir])

        return {
            'root_dir' : initialize.root_dir,
            'source_URL' : initialize.source_URL,
            'local_data_file' : initialize.local_data_file,
            'unzip_dir' : initialize.unzip_dir
        }

In [9]:
class DataIngestionTask:
    def __init__(self, config):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config['local_data_file']):
            source_url = self.config['source_URL']
            file_id = source_url.split('/')[-2]
            filename, headers = request.urlretrieve(
                url=f"https://drive.google.com/uc?id={file_id}",
                filename=self.config['local_data_file']
            )
            logger.info(f"filename: {filename} download with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config['local_data_file']))}")

    def extract_zip_file(self):
        unzip_path = self.config['unzip_dir']
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config['local_data_file'], 'r') as zip_f:
            zip_f.extractall(unzip_path)

In [10]:
config_manager = ConfigurationManagerTask()
data_ingestion_config = config_manager.get_data_ingestion()
data_ingestion = DataIngestionTask(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[ 2023-12-23 18:54:10,493:  INFO:  common: 33:  YAML file 'config\config.yaml' loaded successfully]
[ 2023-12-23 18:54:10,503:  INFO:  common: 53:  Created directory at : artifacts]
[ 2023-12-23 18:54:10,507:  INFO:  common: 53:  Created directory at : artifacts/data_ingestion]
[ 2023-12-23 18:54:13,018:  INFO:  1561642007: 13:  filename: artifacts/data_ingestion/data.zip download with following info:
X-GUploader-UploadID: ABPtcPrnPYz1BFKt8lLKQgLJznnXHahx-c7hxKoHNNau7lN-xPSgI1d9L0IFDVDrpQF_HKLRupQ558Vo2Q
X-Content-Type-Options: nosniff
Content-Type: application/x-zip-compressed
Content-Disposition: attachment; filename="WineQT.zip"; filename*=UTF-8''WineQT.zip
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: false
Access-Control-Allow-Headers: Accept, Accept-Language, Authorization, Cache-Control, Content-Disposition, Content-Encoding, Content-Language, Content-Length, Content-MD5, Content-Range, Content-Type, Date, developer-token, financial-institution-id, X-Goog-Sn-M